This model generates the dose profile from an arbitrary activation profile. It is trained on patient-specific data from a set of 948 pencil beams.

In [1]:
### ONLY FOR GOOGLE COLAB
# Mounting google drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/MyDrive/Colab Notebooks/prototwin/deep-learning-dose-activity-dictionary")
!pip install livelossplot
###

from train_model import train
from test_model import test
from utils import set_seed, DoseActivityDataset, plot_slices, plot_ddp
seed = 42
set_seed(seed)
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Normalize, RandAugment, RandomRotation, GaussianBlur, RandomHorizontalFlip, RandomVerticalFlip
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Creating the dataset
input_dir = "data/dataset_1/input"
output_dir = "data/dataset_1/output"

# Statistics of the dataset (previously found for the entire Prostate dataset)
mean_input = 0.002942
std_input = 0.036942
max_input = 1.977781
min_input = 0.0

mean_output = 0.00000057475
std_output = 0.00000662656
max_output = 0.00060621166
min_output = 0.0


# Transformations
input_transform = Compose([
    # GaussianBlur(kernel_size=3, sigma=1.0),
    Normalize(mean_input, std_input)
])

output_transform = Compose([
    Normalize(mean_output, std_output)
])

# joint_transform = Compose([
#     RandomRotation(5),
#     RandAugment(magnitude=2),
#     RandomHorizontalFlip(),
#     RandomVerticalFlip()
# ])

# Create dataset applying the transforms
dataset = DoseActivityDataset(input_dir=input_dir, output_dir=output_dir,
                              input_transform=input_transform, output_transform=output_transform,
                              num_samples=25)
########## change num_samples

# Split dataset into 70% training, 20% validation, 10% testing
train_size = int(0.7 * len(dataset))
test_size = int(0.2 * len(dataset))
val_size = len(dataset) - train_size - test_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

In [3]:
# Create DataLoaders for training
batch_size = 8  # Largest batch size without running out of memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [4]:
from models.models import R2AttUNet
# Create the model
model = R2AttUNet().to(device)

In [5]:
model_dir = 'models/trained-models/R2AttUNet-v1.pth'
timing_dir = 'models/training-times/training-time-R2AttUNet-v1.txt'
n_epochs = 1  ###############
trained_model = train(model, train_loader, val_loader, epochs=n_epochs,
                      model_dir=model_dir, timing_dir=timing_dir)

  0%|          | 0/3 [00:02<?, ?it/s]


OutOfMemoryError: ignored

In [ ]:
# Loading the trained model
model_dir = "models/trained-models/R2AttUNet-v1.pth"
trained_model = torch.load(model_dir, map_location=torch.device(device))

In [ ]:
# Plotting slices of the dose
plot_slices(trained_model, val_loader, device, mean_input=mean_input, std_input=std_input,
            mean_output=mean_output, std_output=std_output,
            save_plot_dir = "images/R2AttUNet-v1-sample.png")

In [ ]:
# Plotting the dose-depth profiles
save_plot_dir = "images/R2AttUNet-v1-ddp.png"
plot_ddp(trained_model, train_loader, device, mean_output=mean_output,
         std_output=std_output, save_plot_dir = save_plot_dir)

In [ ]:
results_dir = 'models/test-results/R2AttUNet-v1-results.txt'
test(trained_model, test_loader, device, results_dir=results_dir)